# GIFT-Riemann Phase 2 : Test d'Universalité L-Functions

## Objectif

Tester si la récurrence GIFT et ses ratios calibrés sont **universels** (valables pour d'autres L-functions) ou **spécifiques** à ζ(s).

### Questions Clés

| Question | Si OUI | Si NON |
|----------|--------|--------|
| Mêmes lags [5,8,13,27] marchent ? | Structure universelle | Spécifique à ζ(s) |
| Mêmes ratios GIFT ? | GIFT = foncteur arithmétique | Ratios dépendent de L |
| b₃ = 77 domine ? | Cohomologie L² universelle | Autre structure |

### L-Functions Testées

1. **L(s, χ₄)** : Caractère de Dirichlet mod 4 (quadratique)
2. **L(s, χ₃)** : Caractère mod 3
3. **L(s, χ₅)** : Caractère mod 5
4. **Courbes elliptiques** : Si données disponibles

In [ ]:
import numpy as np
import json
from typing import List, Tuple, Dict

# GPU Setup
try:
    import cupy as cp
    GPU_AVAILABLE = True
    xp = cp
    print(f"✅ CuPy GPU disponible")
except ImportError:
    GPU_AVAILABLE = False
    xp = np
    print("⚠️ CuPy non disponible - mode CPU")

print(f"Backend: {'GPU' if GPU_AVAILABLE else 'CPU'}")

## 1. Chargement des Données

### Option A : Upload de fichiers
Uploadez vos fichiers de zéros pour chaque L-function.

### Option B : Génération via mpmath
Génère les premiers zéros (plus lent mais autonome).

### Option C : LMFDB
Téléchargez depuis https://www.lmfdb.org/zeros/

In [ ]:
def load_zeros_from_file(filepath: str) -> np.ndarray:
    """Charge les zéros depuis un fichier."""
    zeros = []
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and not line.startswith('V'):
                    try:
                        val = float(line.split()[0])
                        if val > 0:
                            zeros.append(val)
                    except:
                        continue
        print(f"Chargé {len(zeros)} zéros depuis {filepath}")
        return np.array(sorted(zeros))
    except FileNotFoundError:
        print(f"Fichier non trouvé: {filepath}")
        return None

def generate_dirichlet_zeros(chi_mod: int, n_zeros: int = 1000) -> np.ndarray:
    """
    Génère les zéros d'une L-function de Dirichlet via mpmath.
    LENT - utiliser seulement si pas de données préchargées.
    """
    try:
        from mpmath import mp, dirichlet, nzeros
        mp.dps = 30
        
        print(f"Génération des {n_zeros} premiers zéros de L(s, χ_{chi_mod})...")
        print("(Ceci peut prendre plusieurs minutes)")
        
        # Pour Dirichlet, utiliser siegelz ou calcul direct
        # Placeholder - mpmath n'a pas de fonction directe pour tous les χ
        print("⚠️ Génération automatique limitée - uploadez des données LMFDB")
        return None
        
    except ImportError:
        print("mpmath non disponible")
        return None

In [ ]:
# ============================================================
# CHARGEMENT DES DONNÉES
# ============================================================

# Zéros de Riemann (référence)
zeta_zeros = load_zeros_from_file('zeros1')

# Zéros de L-functions Dirichlet (à uploader)
# Format attendu: un zéro par ligne (partie imaginaire γ)

# Essayer de charger des fichiers existants
L_data = {
    'zeta': zeta_zeros,
}

# Tentative de chargement d'autres L-functions
for name, filepath in [('chi_3', 'zeros_chi3.txt'), 
                        ('chi_4', 'zeros_chi4.txt'),
                        ('chi_5', 'zeros_chi5.txt')]:
    data = load_zeros_from_file(filepath)
    if data is not None:
        L_data[name] = data

print(f"\n📊 L-functions chargées: {list(L_data.keys())}")

In [ ]:
# Upload interactif (Colab)
def upload_L_zeros():
    """Upload interactif pour Colab."""
    try:
        from google.colab import files
        print("Uploadez vos fichiers de zéros L-function...")
        print("Nommez-les: zeros_chi3.txt, zeros_chi4.txt, etc.")
        uploaded = files.upload()
        
        for filename in uploaded:
            zeros = []
            content = uploaded[filename].decode('utf-8')
            for line in content.split('\n'):
                line = line.strip()
                if line and not line.startswith('#'):
                    try:
                        zeros.append(float(line.split()[0]))
                    except:
                        continue
            
            name = filename.replace('zeros_', '').replace('.txt', '')
            L_data[name] = np.array(sorted(zeros))
            print(f"  {name}: {len(zeros)} zéros")
            
    except ImportError:
        print("Pas sur Colab - utilisez load_zeros_from_file()")

# Décommenter pour upload:
# upload_L_zeros()

## 2. Fonctions d'Analyse

In [ ]:
def local_spacing(T: np.ndarray, conductor: float = 1.0) -> np.ndarray:
    """
    Espacement local pour L-function.
    Pour Dirichlet: Δ(T) ≈ 2π / log(qT/2π) où q = conducteur
    """
    safe_T = np.maximum(T, 2 * np.pi / conductor + 0.1)
    return 2 * np.pi / np.log(conductor * safe_T / (2 * np.pi))


def fit_recurrence(gamma: np.ndarray, lags: List[int], 
                   start: int = None, end: int = None) -> Tuple[np.ndarray, float]:
    """
    Fit récurrence linéaire.
    Retourne: (coefficients, erreur_unfolded_moyenne)
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    n_points = end - start
    n_params = len(lags) + 1
    
    # Build matrices
    X = np.zeros((n_points, n_params))
    for i, lag in enumerate(lags):
        X[:, i] = gamma[start - lag:end - lag]
    X[:, -1] = 1.0
    
    y = gamma[start:end]
    
    # Least squares
    coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    # Error
    y_pred = X @ coeffs
    errors = np.abs(y_pred - y)
    spacings = local_spacing(y)
    errors_unf = errors / spacings
    
    return coeffs, np.mean(errors_unf)


def analyze_L_function(name: str, gamma: np.ndarray, 
                       reference_lags: List[int] = [5, 8, 13, 27],
                       stable_start_ratio: float = 0.7) -> Dict:
    """
    Analyse complète d'une L-function.
    """
    if gamma is None or len(gamma) < 100:
        print(f"⚠️ {name}: pas assez de données")
        return None
    
    N = len(gamma)
    max_lag = max(reference_lags)
    stable_start = max(max_lag + 1, int(N * stable_start_ratio))
    
    print(f"\n{'='*60}")
    print(f"ANALYSE: {name} ({N} zéros)")
    print(f"{'='*60}")
    
    # 1. Test avec lags GIFT
    print(f"\n📊 Test lags GIFT {reference_lags}:")
    coeffs_gift, error_gift = fit_recurrence(gamma, reference_lags, stable_start, N)
    print(f"   Erreur unfolded: {error_gift:.4f} spacings")
    
    # Coefficients
    print(f"\n   Coefficients (n > {stable_start}):")
    for i, lag in enumerate(reference_lags):
        print(f"     a_{lag} = {coeffs_gift[i]:.6f}")
    print(f"     c = {coeffs_gift[-1]:.6f}")
    
    # 2. Comparaison avec ratios GIFT calibrés
    gift_calibrated = {
        5: ('rank(E₈)/b₃', 8/77),
        8: ('Weyl/dim(J₃𝕆)', 5/27),
        13: ('rank(E₈)²/dim(E₈)', 64/248),
        27: ('(27+7)/b₃', 34/77),
        'c': ('(b₃+14)/dim(K₇)', 91/7),
    }
    
    print(f"\n📈 Comparaison GIFT calibré (depuis ζ):")
    print(f"   {'Coeff':<8} {'Mesuré':>12} {'GIFT ζ':>12} {'Écart':>10}")
    print(f"   {'-'*45}")
    
    matches = []
    for i, lag in enumerate(reference_lags):
        measured = coeffs_gift[i]
        _, gift_val = gift_calibrated[lag]
        ecart = abs(measured - gift_val) / abs(gift_val) * 100
        matches.append(ecart)
        print(f"   a_{lag:<5} {measured:>12.6f} {gift_val:>12.6f} {ecart:>9.1f}%")
    
    _, gift_c = gift_calibrated['c']
    ecart_c = abs(coeffs_gift[-1] - gift_c) / abs(gift_c) * 100
    matches.append(ecart_c)
    print(f"   {'c':<6} {coeffs_gift[-1]:>12.6f} {gift_c:>12.6f} {ecart_c:>9.1f}%")
    
    # 3. Test avec lags alternatifs
    print(f"\n🔍 Test lags alternatifs:")
    alt_lags = [
        [1, 2, 3, 4],
        [1, 2, 3, 4, 8],
        [1, 2, 3, 4, 8, 27],
        [3, 5, 8, 13],
    ]
    
    for lags in alt_lags:
        if max(lags) < N - stable_start:
            _, err = fit_recurrence(gamma, lags, stable_start, N)
            print(f"   {str(lags):<25} → {err:.4f} spacings")
    
    # 4. Verdict
    avg_match = np.mean(matches)
    print(f"\n🎯 Verdict {name}:")
    if avg_match < 20:
        print(f"   ✅ MATCH GIFT ({avg_match:.1f}% écart moyen)")
        print(f"   → Mêmes ratios que ζ(s)!")
        verdict = "MATCH"
    elif avg_match < 50:
        print(f"   ⚠️ PARTIEL ({avg_match:.1f}% écart moyen)")
        verdict = "PARTIAL"
    else:
        print(f"   ❌ DIFFÉRENT ({avg_match:.1f}% écart moyen)")
        print(f"   → Ratios spécifiques à cette L-function")
        verdict = "DIFFERENT"
    
    return {
        'name': name,
        'n_zeros': N,
        'coefficients': coeffs_gift.tolist(),
        'error_unfolded': error_gift,
        'gift_matches': matches,
        'avg_match_pct': avg_match,
        'verdict': verdict
    }

## 3. Analyse de Toutes les L-Functions

In [ ]:
# Analyse de toutes les L-functions chargées
results = {}

for name, gamma in L_data.items():
    result = analyze_L_function(name, gamma)
    if result:
        results[name] = result

## 4. Comparaison Globale

In [ ]:
def global_comparison(results: Dict) -> Dict:
    """
    Comparaison globale de toutes les L-functions.
    """
    if len(results) < 2:
        print("⚠️ Besoin d'au moins 2 L-functions pour comparer")
        print("   Uploadez des zéros de Dirichlet L-functions")
        return None
    
    print("\n" + "="*70)
    print("COMPARAISON GLOBALE : UNIVERSALITÉ DES RATIOS GIFT")
    print("="*70)
    
    # Tableau comparatif
    print(f"\n{'L-function':<15} {'N zeros':>10} {'Erreur':>10} {'Match GIFT':>12} {'Verdict':<12}")
    print("-" * 65)
    
    for name, r in results.items():
        print(f"{name:<15} {r['n_zeros']:>10} {r['error_unfolded']:>10.4f} {r['avg_match_pct']:>11.1f}% {r['verdict']:<12}")
    
    # Coefficients comparés
    print(f"\n📊 Coefficients par L-function:")
    lags = [5, 8, 13, 27]
    
    header = f"{'Coeff':<8}" + "".join([f"{name:>12}" for name in results.keys()])
    print(header)
    print("-" * len(header))
    
    for i, lag in enumerate(lags):
        row = f"a_{lag:<5}"
        for name, r in results.items():
            row += f"{r['coefficients'][i]:>12.4f}"
        print(row)
    
    row = "c     "
    for name, r in results.items():
        row += f"{r['coefficients'][-1]:>12.4f}"
    print(row)
    
    # Verdict global
    verdicts = [r['verdict'] for r in results.values()]
    n_match = verdicts.count('MATCH')
    n_partial = verdicts.count('PARTIAL')
    n_diff = verdicts.count('DIFFERENT')
    
    print(f"\n🎯 VERDICT GLOBAL:")
    print(f"   MATCH: {n_match}, PARTIAL: {n_partial}, DIFFERENT: {n_diff}")
    
    if n_match == len(results):
        print(f"\n   🔥 UNIVERSALITÉ CONFIRMÉE!")
        print(f"   → Les ratios GIFT sont les mêmes pour toutes les L-functions")
        print(f"   → GIFT encode une structure arithmétique universelle")
        global_verdict = "UNIVERSAL"
    elif n_match > 0:
        print(f"\n   ⚠️ UNIVERSALITÉ PARTIELLE")
        print(f"   → Certaines L-functions matchent, d'autres non")
        global_verdict = "PARTIAL_UNIVERSAL"
    else:
        print(f"\n   ❌ SPÉCIFICITÉ À ζ(s)")
        print(f"   → Les ratios GIFT sont spécifiques à la fonction zeta")
        global_verdict = "ZETA_SPECIFIC"
    
    return {
        'global_verdict': global_verdict,
        'n_match': n_match,
        'n_partial': n_partial,
        'n_different': n_diff,
        'results': results
    }

# Run comparison
comparison = global_comparison(results)

## 5. Recherche de Ratios Spécifiques

Si les ratios diffèrent, cherchons quels ratios GIFT correspondent.

In [ ]:
# GIFT constants
GIFT = {
    'dim_G2': 14, 'b2': 21, 'b3': 77, 'H_star': 99, 'dim_K7': 7,
    'rank_E8': 8, 'dim_E8': 248, 'fund_E7': 56, 'dim_J3O': 27,
    'Weyl': 5, 'N_gen': 3, 'h_G2': 6, 'p2': 2,
}

def find_gift_ratios(target: float, tol: float = 0.1) -> List[Tuple[str, float]]:
    """Trouve des ratios GIFT proches de la valeur cible."""
    matches = []
    
    for n1, v1 in GIFT.items():
        for n2, v2 in GIFT.items():
            if v2 != 0:
                ratio = v1 / v2
                if abs(target) > 0.001 and abs(ratio - target) / abs(target) < tol:
                    matches.append((f"{n1}/{n2}", ratio))
    
    # Trier par proximité
    matches.sort(key=lambda x: abs(x[1] - target))
    return matches[:5]


def reverse_engineer_ratios(results: Dict):
    """
    Pour chaque L-function, cherche les ratios GIFT correspondants.
    """
    print("\n" + "="*70)
    print("REVERSE ENGINEERING: RATIOS GIFT PAR L-FUNCTION")
    print("="*70)
    
    for name, r in results.items():
        print(f"\n📊 {name}:")
        
        lags = [5, 8, 13, 27]
        for i, lag in enumerate(lags):
            coeff = r['coefficients'][i]
            matches = find_gift_ratios(coeff)
            match_str = ", ".join([f"{m[0]}={m[1]:.3f}" for m in matches[:2]]) if matches else "?"
            print(f"   a_{lag} = {coeff:.4f} → {match_str}")
        
        c = r['coefficients'][-1]
        matches = find_gift_ratios(c)
        match_str = ", ".join([f"{m[0]}={m[1]:.3f}" for m in matches[:2]]) if matches else "?"
        print(f"   c = {c:.4f} → {match_str}")

# Run reverse engineering
reverse_engineer_ratios(results)

## 6. Export des Résultats

In [ ]:
# Export JSON
def export_results(results: Dict, comparison: Dict):
    """Export des résultats en JSON."""
    
    output = {
        'phase': 'Phase 2 - L-functions Universality Test',
        'gift_reference': {
            'lags': [5, 8, 13, 27],
            'calibrated_ratios': {
                'a_5': '8/77 = rank(E₈)/b₃',
                'a_8': '5/27 = Weyl/dim(J₃𝕆)',
                'a_13': '64/248 = rank(E₈)²/dim(E₈)',
                'a_27': '34/77 = (27+7)/b₃',
                'c': '91/7 = (b₃+14)/dim(K₇)'
            }
        },
        'results_by_L_function': {},
        'global_comparison': comparison
    }
    
    for name, r in results.items():
        output['results_by_L_function'][name] = {
            'n_zeros': r['n_zeros'],
            'error_unfolded': r['error_unfolded'],
            'coefficients': {
                'a_5': r['coefficients'][0],
                'a_8': r['coefficients'][1],
                'a_13': r['coefficients'][2],
                'a_27': r['coefficients'][3],
                'c': r['coefficients'][4]
            },
            'gift_match_pct': r['avg_match_pct'],
            'verdict': r['verdict']
        }
    
    with open('phase2_L_functions_results.json', 'w') as f:
        json.dump(output, f, indent=2)
    
    print("\n💾 Résultats sauvegardés dans phase2_L_functions_results.json")
    print("\n" + "="*70)
    print("EXPORT JSON")
    print("="*70)
    print(json.dumps(output, indent=2))
    
    return output

# Export
if comparison:
    final_output = export_results(results, comparison)
else:
    print("\n⚠️ Uploadez des données de L-functions pour l'analyse complète")

## 7. Instructions pour Obtenir des Données

### LMFDB (recommandé)

1. Aller sur https://www.lmfdb.org/L/
2. Chercher "Dirichlet L-functions"
3. Télécharger les zéros pour différents caractères

### Exemples de caractères intéressants

| Caractère | Conducteur | Propriété |
|-----------|------------|----------|
| χ₄ (Kronecker) | 4 | Quadratique, lié à i |
| χ₃ | 3 | Cubique |
| χ₅ | 5 | = Weyl dans GIFT! |
| χ₈ | 8 | = rank(E₈) dans GIFT! |

### Format attendu

```
# Fichier zeros_chi4.txt
6.020948
10.243749
12.578617
...
```

Un zéro (partie imaginaire γ) par ligne.